In [1]:
# This file will be used to prepare the data for the graphical models (or strategy structure predictions)
import pickle, pdb
import sys
sys.path.append('..') # for utils
sys.path.append('../..') # for cocoa
import utils
from collections import defaultdict as ddict
import numpy as np
from cocoa_folder.cocoa.core.dataset import Example
from cocoa_folder.craigslistbargain.parse_dialogue import parse_example
from cocoa_folder.craigslistbargain.core.price_tracker import PriceTracker
from cocoa_folder.craigslistbargain.model.generator import Templates
from cocoa_folder.craigslistbargain.core.scenario import Scenario

[nltk_data] Downloading package punkt to /home/rjoshi2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/rjoshi2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/rjoshi2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
price_tracker = PriceTracker(utils.PROJ_DIR + "cocoa_folder/craigslistbargain/price_tracker.pkl")
templates = Templates()

In [4]:
data_w_strategies = pickle.load(open(utils.PROJ_DIR + 'data/negotiation_data/data_w_strategies_outcomes.pkl', 'rb'))

In [5]:
# FULL has normalize_price = False
# FULL_Yiheng has normalize_price = True

In [6]:
norm_price = True

In [7]:
def get_strategies(data_point):
    '''
    Takes data point and returns list of negotiation strategies, and dialogue acts
    '''
    utterances = parse_example(Example.from_dict(data_point, Scenario), price_tracker, templates) # Taken from Yiheng's work
    utterances = utterances[2:] # skip first two
    #agent_list = [x['agent'] for x in data_point['events'] if type(x['data']) == str]
    agent_list = [-1]
    strategy_list = [['<start>']]    # start + len(events) #  +[x['strategies'] for x in data_point['events'] if type(x['data']) == str]
    utterance_list = ['<start>'] # start + len(events)
    dial_act_list = ['<start>']    # start + len(events)
    last_strat = []
    for i, utt in enumerate(utterances):
        if utt.text == None:
            text = '<' + str(data_point['events'][i]['action']) + '>' # offer, accept, reject, quit
            agent = data_point['events'][i]['agent']
            dial_act = '<' + utt.lf.to_dict()['intent'] + '>'
            if dial_act != '<start>': strats = last_strat             # copy last strategy for end portions
            else:                     strats = []
        else:
            text = utils.normalizeString(utt.text, utt.lf.to_dict(), data_point["scenario"], normalize_price = norm_price)
            strats = data_point['events'][i]['strategies']
            last_strat = strats
            agent = data_point['events'][i]['agent']
            dial_act = utt.lf.to_dict()['intent']
        utterance_list.append(text)
        dial_act_list.append(dial_act)
        agent_list.append(agent)
        strategy_list.append(strats)
    assert len(utterance_list) == len(data_point['events']) + 1, pdb.set_trace()
    assert len(utterance_list) == len(strategy_list), pdb.set_trace()
    assert len(utterance_list) == len(agent_list), pdb.set_trace()
    assert len(utterance_list) == len(dial_act_list), pdb.set_trace()
    return agent_list, utterance_list, strategy_list, dial_act_list
    
#    return [(x['agent'], x['data'], x['strategies']) for x in data_point['events'] if type(x['data']) == str]

In [8]:
# full_data = [] # list of conversations
# for i, data_point in enumerate(train_data):
#     agent_list, utterance_list, strategies_list, dial_act_list = get_strategies(data_point)
#     ratio = utils.getRatio(data_point)
#     full_data.append({'agent_list': agent_list,
#                      'utterance': utterance_list,
#                      'strategies': strategies_list,
#                      'ratio': ratio,
#                      'dial_acts': dial_act_list
#                      })
# print (len(full_data))
# print (full_data[76])

full_data_all = {}
for dtype in data_w_strategies:
    print ('Dtype : ', dtype)
    data = data_w_strategies[dtype]
    full_data_list = []
    for i, data_point in enumerate(data):
        agent_list, utterance_list, strategies_list, dial_act_list = get_strategies(data_point)
        ratio = utils.getRatio(data_point)
        uuid  = data_point['uuid']
        scene = (data_point["scenario"]["kbs"][1]["item"]["Category"] + " " + " ".join(data_point["scenario"]["kbs"][1]["item"]["Description"])+ " " + data_point["scenario"]["kbs"][1]["item"]["Title"])
        scene = utils.normalizeString(scene, {"price":None}, data_point["scenario"], normalize_price = norm_price) 
        full_data_list.append({'agent_list': agent_list,
                     'utterance': utterance_list,
                     'strategies': strategies_list,
                     'ratio': ratio,
                     'dial_acts': dial_act_list,
                     'uuid': uuid,
                     'scene': scene,
                     'raw_data': data_point
                     })
    #print (len(full_data_list))
    #print (full_data_list[76])
    full_data_all[dtype] = full_data_list


Dtype :  train
Dtype :  test
Dtype :  dev


## Only use full_data_all from now on

In [9]:
# See Frequent strategies
strategies_freq = {}
all_strategies = {}
dial_act_freq = {}
all_dial_act = {}
for dtype in full_data_all:
    print ('DTYPE : ', dtype)
    data = full_data_all[dtype]
    strategies_freq_ = ddict(int)
    dial_act_freq_   = ddict(int)
    all_strategies_ = set()
    all_dial_act_   = set()
    for i, data_point in enumerate(data):
        for conv in data_point['strategies']:
            for strategy in conv:
                strategies_freq_[strategy] += 1
                all_strategies_.add(strategy)
        for da in data_point['dial_acts']:
            dial_act_freq_[da] += 1
            all_dial_act_.add(da)
    print (strategies_freq_)
    print (all_strategies_)
    print (f'Total {len(all_strategies_)} strategies')
    print (dial_act_freq_)
    print (all_dial_act_)
    print (f'Total {len(all_dial_act_)} dial acts')
    strategies_freq[dtype] = strategies_freq_
    all_strategies[dtype]  = all_strategies_
    dial_act_freq[dtype] = dial_act_freq_
    all_dial_act[dtype]  = all_dial_act_

DTYPE :  train
defaultdict(<class 'int'>, {'<start>': 5383, 'buyer_pos_sentiment': 12710, 'first_person_singular_count_buyer': 14341, 'third_person_singular_buyer': 8122, 'buyer_propose': 5249, 'hedge_count_buyer': 7023, 'assertive_count_buyer': 2044, 'politeness_buyer_greet': 3596, 'first_person_singular_count_seller': 11780, 'third_person_singular_seller': 8878, 'number_of_diff_dic_pos': 18610, 'number_of_diff_dic_neg': 10402, 'personal_concern_seller': 9135, 'liwc_certainty': 2530, 'seller_propose': 3200, 'hedge_count_seller': 5204, 'assertive_count_seller': 2393, 'first_person_plural_count_seller': 1843, 'politeness_seller_greet': 3043, 'factive_count_buyer': 1666, 'factive_count_seller': 1763, 'third_person_plural_buyer': 744, 'seller_neg_sentiment': 2587, 'seller_pos_sentiment': 12152, 'liwc_informal': 2396, 'who_propose': 1009, 'first_person_plural_count_buyer': 1033, 'politeness_seller_gratitude': 1073, 'buyer_neg_sentiment': 1093, 'politeness_seller_please': 60, 'politeness_bu

In [10]:
# THIS FEATURE MAP IS USED BY YIHENG
recommendation_feature_mapping = {"seller_neg_sentiment":0,"seller_pos_sentiment":1,
                                  "buyer_neg_sentiment":2,"buyer_pos_sentiment":3,
                                  "first_person_plural_count_seller":4,"first_person_singular_count_seller":5,
                                  "first_person_plural_count_buyer":6,"first_person_singular_count_buyer":7,
                                  "third_person_singular_seller":8,"third_person_plural_seller":9,
                                  "third_person_singular_buyer":10,"third_person_plural_buyer":11,
                                  "number_of_diff_dic_pos":12,"number_of_diff_dic_neg":13,
                                  "buyer_propose":14,"seller_propose":15,
                                  "hedge_count_seller":16,"hedge_count_buyer":17,
                                  "assertive_count_seller":18,"assertive_count_buyer":19,
                                  "factive_count_seller":20,"factive_count_buyer":21,
                                  "who_propose":22,"seller_trade_in":23,
                                  "personal_concern_seller":24,"sg_concern":25,
                                  "liwc_certainty":26,"liwc_informal":27,
                                  "politeness_seller_please":28,"politeness_seller_gratitude":29,
                                  "politeness_seller_please_s":30,
                                  "ap_des":31,"ap_pata":32,"ap_infer":33,
                                  "family":34,"friend":35,
                                  "politeness_buyer_please":36,"politeness_buyer_gratitude":37,
                                  "politeness_buyer_please_s":38,
                                  "politeness_seller_greet":39,"politeness_buyer_greet":40}

In [11]:
# See what all strategies are there in the feature mapping and not there in all types of data
yiheng_strategies = set([k for k in recommendation_feature_mapping])
for dtype in all_strategies:
    print (dtype)
    all_strategies_ = all_strategies[dtype]
    print (all_strategies_ - yiheng_strategies)
    print (yiheng_strategies - all_strategies_)

train
{'<start>'}
{'ap_pata', 'ap_infer', 'sg_concern', 'ap_des'}
test
{'<start>'}
{'politeness_buyer_please_s', 'sg_concern', 'ap_infer', 'politeness_seller_please_s', 'ap_des', 'ap_pata'}
dev
{'<start>'}
{'ap_pata', 'ap_infer', 'sg_concern', 'ap_des'}


In [12]:
# Map recommendation feature mapping to a uniform_strategy_mapping
# This uniform mapping is without sellers or buyers
# For exampel : {buyer_propose : propose, seller_propose : propose}
recommended2uniform_strategymapping = {}
idx = 0
for strategy in recommendation_feature_mapping:
    new_strategy = strategy.replace('_seller', '').replace('_buyer', '').replace('seller_', '').replace('buyer_', '').replace('please_s', 'please')
    recommended2uniform_strategymapping[strategy] = new_strategy
print (recommended2uniform_strategymapping)

{'seller_neg_sentiment': 'neg_sentiment', 'seller_pos_sentiment': 'pos_sentiment', 'buyer_neg_sentiment': 'neg_sentiment', 'buyer_pos_sentiment': 'pos_sentiment', 'first_person_plural_count_seller': 'first_person_plural_count', 'first_person_singular_count_seller': 'first_person_singular_count', 'first_person_plural_count_buyer': 'first_person_plural_count', 'first_person_singular_count_buyer': 'first_person_singular_count', 'third_person_singular_seller': 'third_person_singular', 'third_person_plural_seller': 'third_person_plural', 'third_person_singular_buyer': 'third_person_singular', 'third_person_plural_buyer': 'third_person_plural', 'number_of_diff_dic_pos': 'number_of_diff_dic_pos', 'number_of_diff_dic_neg': 'number_of_diff_dic_neg', 'buyer_propose': 'propose', 'seller_propose': 'propose', 'hedge_count_seller': 'hedge_count', 'hedge_count_buyer': 'hedge_count', 'assertive_count_seller': 'assertive_count', 'assertive_count_buyer': 'assertive_count', 'factive_count_seller': 'facti

In [13]:
recommended2uniform_strategymapping['<start>'] = '<start>'

In [14]:
recommended2uniform_strategymapping

{'seller_neg_sentiment': 'neg_sentiment',
 'seller_pos_sentiment': 'pos_sentiment',
 'buyer_neg_sentiment': 'neg_sentiment',
 'buyer_pos_sentiment': 'pos_sentiment',
 'first_person_plural_count_seller': 'first_person_plural_count',
 'first_person_singular_count_seller': 'first_person_singular_count',
 'first_person_plural_count_buyer': 'first_person_plural_count',
 'first_person_singular_count_buyer': 'first_person_singular_count',
 'third_person_singular_seller': 'third_person_singular',
 'third_person_plural_seller': 'third_person_plural',
 'third_person_singular_buyer': 'third_person_singular',
 'third_person_plural_buyer': 'third_person_plural',
 'number_of_diff_dic_pos': 'number_of_diff_dic_pos',
 'number_of_diff_dic_neg': 'number_of_diff_dic_neg',
 'buyer_propose': 'propose',
 'seller_propose': 'propose',
 'hedge_count_seller': 'hedge_count',
 'hedge_count_buyer': 'hedge_count',
 'assertive_count_seller': 'assertive_count',
 'assertive_count_buyer': 'assertive_count',
 'factive_c

In [15]:
# Unique types of uniform_strategies
print (len(set([v for k,v in recommended2uniform_strategymapping.items()])))

27


In [16]:
uniform_feature_mapping = {}
skip_strategies = ['who_propose'] + list(yiheng_strategies - all_strategies['dev'])
print ('Skip strategies : ', skip_strategies)
uniform_strategies = set([v for k,v in recommended2uniform_strategymapping.items()])
idx = 0
for i, item in enumerate(uniform_strategies):
    if item in skip_strategies:
        continue
    uniform_feature_mapping[item] = idx
    idx += 1
print (uniform_feature_mapping)

Skip strategies :  ['who_propose', 'ap_pata', 'ap_infer', 'sg_concern', 'ap_des']
{'family': 0, 'third_person_plural': 1, 'friend': 2, 'trade_in': 3, 'first_person_plural_count': 4, 'politeness_greet': 5, 'neg_sentiment': 6, 'politeness_gratitude': 7, 'propose': 8, 'hedge_count': 9, '<start>': 10, 'number_of_diff_dic_pos': 11, 'first_person_singular_count': 12, 'assertive_count': 13, 'liwc_informal': 14, 'pos_sentiment': 15, 'personal_concern': 16, 'third_person_singular': 17, 'politeness_please': 18, 'number_of_diff_dic_neg': 19, 'factive_count': 20, 'liwc_certainty': 21}


In [17]:
# See frequency of uniform_strategies
strategies_freq = ddict(int)
for i, data_point in enumerate(full_data_all['train']):
    for conv in data_point['strategies']:
        for strategy in conv:
#             if strategy == '<start>':
#                 uniform_strategy = strategy
#             else:
            uniform_strategy = recommended2uniform_strategymapping[strategy]
            strategies_freq[uniform_strategy] += 1
from pprint import pprint
#pprint (sorted(strategies_freq.items(), key = lambda x: x[1], reverse = True))
tot = np.sum([v for k,v in strategies_freq.items()])
print (tot)
for k,v in (sorted(strategies_freq.items(), key = lambda x: x[1], reverse = True)):
    print ("%30s %.4f %d" % (k, v / tot, v))
pprint (f'Total {len(strategies_freq)-1} strategies') # -1 for who_propose

165682
   first_person_singular_count 0.1577 26121
                 pos_sentiment 0.1501 24862
        number_of_diff_dic_pos 0.1123 18610
         third_person_singular 0.1026 17000
                   hedge_count 0.0738 12227
        number_of_diff_dic_neg 0.0628 10402
              personal_concern 0.0551 9135
                       propose 0.0510 8449
              politeness_greet 0.0401 6639
                       <start> 0.0325 5383
               assertive_count 0.0268 4437
                 neg_sentiment 0.0222 3680
                 factive_count 0.0207 3429
          politeness_gratitude 0.0191 3171
     first_person_plural_count 0.0174 2876
                liwc_certainty 0.0153 2530
                 liwc_informal 0.0145 2396
           third_person_plural 0.0104 1721
                   who_propose 0.0061 1009
                      trade_in 0.0053 883
             politeness_please 0.0022 372
                        family 0.0012 201
                        friend 0.0009 149
'T

In [18]:
# See frequency of dial_acts
da_freq = dial_act_freq['train']
from pprint import pprint
#pprint (sorted(strategies_freq.items(), key = lambda x: x[1], reverse = True))
tot = np.sum([v for k,v in da_freq.items()])
print (tot)
for k,v in (sorted(da_freq.items(), key = lambda x: x[1], reverse = True)):
    print ("%30s %.4f %d" % (k, v / tot, v))
pprint (f"Total {len(all_dial_act['train'])} dialogue acts")

54980
                       unknown 0.1789 9838
                 counter-price 0.1463 8042
                       <start> 0.0979 5383
                       inquiry 0.0950 5224
                    init-price 0.0878 4828
                         intro 0.0863 4743
                       <offer> 0.0847 4657
                      <accept> 0.0743 4084
                        inform 0.0439 2413
                      disagree 0.0379 2085
                         agree 0.0361 1985
                        insist 0.0086 474
                        <quit> 0.0086 474
                   vague-price 0.0073 401
                      <reject> 0.0063 349
'Total 15 dialogue acts'


In [19]:
chosen_strategies_mapping = {}#{k:v for k,v in uniform_feature_mapping.items() if k != '<start>'}
cntr = 0
for k,v in uniform_feature_mapping.items():
    if k!='<start>':
        chosen_strategies_mapping[k] = cntr
        cntr += 1
chosen_strategies_mapping['<start>'] = len(chosen_strategies_mapping)
# skip_strategies = ['who_propose'] + list(yiheng_strategies - all_strategies)
# print ("Skip strategies : ", skip_strategies)
# idx = 0
# for strategy, mapping in uniform_feature_mapping.items():
#     if strategy in skip_strategies:
#         continue
#     chosen_strategies_mapping[strategy] = idx
#     idx += 1
print (chosen_strategies_mapping)
print (len(chosen_strategies_mapping))

{'family': 0, 'third_person_plural': 1, 'friend': 2, 'trade_in': 3, 'first_person_plural_count': 4, 'politeness_greet': 5, 'neg_sentiment': 6, 'politeness_gratitude': 7, 'propose': 8, 'hedge_count': 9, 'number_of_diff_dic_pos': 10, 'first_person_singular_count': 11, 'assertive_count': 12, 'liwc_informal': 13, 'pos_sentiment': 14, 'personal_concern': 15, 'third_person_singular': 16, 'politeness_please': 17, 'number_of_diff_dic_neg': 18, 'factive_count': 19, 'liwc_certainty': 20, '<start>': 21}
22


In [20]:
chosen_dial_act_mapping = {}#{k:i for i, k in enumerate(all_dial_act['train']) if k != '<start>' }
cntr = 0
for k,v in enumerate(all_dial_act['train']):
    if v != '<start>':
        chosen_dial_act_mapping[v] = cntr
        cntr += 1
chosen_dial_act_mapping['<start>'] = len(chosen_dial_act_mapping)
print (chosen_dial_act_mapping)
print (len(chosen_dial_act_mapping))

{'inform': 0, 'init-price': 1, 'inquiry': 2, 'disagree': 3, 'counter-price': 4, 'unknown': 5, '<reject>': 6, '<offer>': 7, 'intro': 8, 'vague-price': 9, '<accept>': 10, '<quit>': 11, 'insist': 12, 'agree': 13, '<start>': 14}
15


In [21]:
# ap_pata, ap_des, ap_infer and sg_concern are strategies that are learned using the classifiers. 
# Yiheng did not have any way to extract these for the data from the code he had access.

In [22]:
# See # of strategies per utterance frequency to see distribution
for dtype in full_data_all:
    print (dtype)
    strategies_perutt_freq = ddict(int)
    for i, data_point in enumerate(full_data_all[dtype]):
        for conv in data_point['strategies']:
            num = len([c for c in conv if recommended2uniform_strategymapping[c] in chosen_strategies_mapping])
            strategies_perutt_freq[num] += 1
    print (sorted(strategies_perutt_freq.items(), key = lambda x: x))
    tot = np.sum([v for k,v in strategies_perutt_freq.items()])
    mean_strategies_per_turn = np.sum([k*v for k,v in strategies_perutt_freq.items()]) / tot
    print ('Mean strategies per utt : ', mean_strategies_per_turn)

train
[(0, 3062), (1, 13552), (2, 10047), (3, 8840), (4, 7222), (5, 5025), (6, 3329), (7, 1958), (8, 1116), (9, 511), (10, 236), (11, 61), (12, 17), (13, 4)]
Mean strategies per utt :  2.9951436886140415
test
[(0, 1059), (1, 1839), (2, 1160), (3, 892), (4, 635), (5, 367), (6, 251), (7, 134), (8, 49), (9, 29), (10, 9), (11, 3)]
Mean strategies per utt :  2.2452154971215186
dev
[(0, 409), (1, 1646), (2, 1159), (3, 1031), (4, 845), (5, 578), (6, 356), (7, 223), (8, 143), (9, 51), (10, 26), (11, 12), (12, 5)]
Mean strategies per utt :  2.942473781616286


In [23]:
# See max and avg turns
for dtype in full_data_all:
    print (dtype)
    max_turns = 0
    avg_turns = 0
    turns_freq = ddict(int)
    for i, data_point in enumerate(full_data_all[dtype]):
        turns = len(data_point['strategies'])
        max_turns = max(max_turns, turns)
        avg_turns += turns
        turns_freq[turns] += 1
    avg_turns /= len(full_data_all[dtype])
    print (f'Max turns are {max_turns}')
    print (f'Avg turns are {avg_turns}')
    print (sorted(turns_freq.items()))

train
Max turns are 47
Avg turns are 10.213635519227196
[(3, 246), (4, 309), (5, 182), (6, 178), (7, 247), (8, 471), (9, 511), (10, 677), (11, 585), (12, 623), (13, 414), (14, 333), (15, 211), (16, 137), (17, 100), (18, 49), (19, 46), (20, 25), (21, 19), (22, 10), (23, 4), (24, 1), (25, 1), (26, 1), (30, 1), (31, 1), (47, 1)]
test
Max turns are 24
Avg turns are 9.797256097560975
[(3, 44), (4, 45), (5, 35), (6, 22), (7, 24), (8, 63), (9, 58), (10, 66), (11, 77), (12, 63), (13, 51), (14, 39), (15, 23), (16, 23), (17, 9), (18, 5), (19, 3), (20, 3), (21, 1), (24, 2)]
dev
Max turns are 28
Avg turns are 10.08398133748056
[(3, 35), (4, 47), (5, 21), (6, 24), (7, 25), (8, 65), (9, 57), (10, 66), (11, 66), (12, 64), (13, 48), (14, 53), (15, 21), (16, 21), (17, 10), (18, 9), (19, 6), (20, 1), (21, 1), (23, 1), (25, 1), (28, 1)]


## Vectorify full_data

In [24]:
# convert data into vector form
def convert_2dlistofstrategies2vector(strategies, agent_list):
    '''
    Takes a list of list of strategies and returns numpy arr 2d numpy arr : conv_len x num_strats
    '''
    strats = np.zeros((len(strategies), len(chosen_strategies_mapping) + 1)) # 1 for agent
    for i, stlist in enumerate(strategies):
        for st in stlist:
            uniform_st = recommended2uniform_strategymapping[st]
            if uniform_st not in chosen_strategies_mapping:
                continue
            strats[i][chosen_strategies_mapping[uniform_st] + 1] = 1    # 1 for leaving space for ageent
        try:
            if agent_list[i] != -1:
                strats[i][0] = agent_list[i]
        except:
            pdb.set_trace()
    return np.array(strats)

# for i, data_point in enumerate(full_data):
#     full_data[i]['strategies_vec'] = convert_2dlistofstrategies2vector(full_data[i]['strategies'], full_data[i]['agent_list'])
# print (full_data[1234])

In [25]:
for dtype in full_data_all:
    print (dtype)
    data = full_data_all[dtype]
    for i in range(len(data)):
        data[i]['strategies_vec'] = convert_2dlistofstrategies2vector(data[i]['strategies'], data[i]['agent_list'])
        data[i]['dial_acts_vec']   = np.array([chosen_dial_act_mapping[d] for d in data[i]['dial_acts']])

train
test
dev


In [26]:
print (full_data_all['train'][1234]['strategies_vec'].shape)
print (full_data_all['train'][1234]['dial_acts_vec'].shape)

(22, 23)
(22,)


In [27]:
strategies2colid = {}
strategies2colid['agent_id'] = 0
for s, idx in chosen_strategies_mapping.items():
    strategies2colid[s] = idx+1 # for agent
print (strategies2colid)
colid2strategies = {v:k for k,v in strategies2colid.items()}
print (colid2strategies)

{'agent_id': 0, 'family': 1, 'third_person_plural': 2, 'friend': 3, 'trade_in': 4, 'first_person_plural_count': 5, 'politeness_greet': 6, 'neg_sentiment': 7, 'politeness_gratitude': 8, 'propose': 9, 'hedge_count': 10, 'number_of_diff_dic_pos': 11, 'first_person_singular_count': 12, 'assertive_count': 13, 'liwc_informal': 14, 'pos_sentiment': 15, 'personal_concern': 16, 'third_person_singular': 17, 'politeness_please': 18, 'number_of_diff_dic_neg': 19, 'factive_count': 20, 'liwc_certainty': 21, '<start>': 22}
{0: 'agent_id', 1: 'family', 2: 'third_person_plural', 3: 'friend', 4: 'trade_in', 5: 'first_person_plural_count', 6: 'politeness_greet', 7: 'neg_sentiment', 8: 'politeness_gratitude', 9: 'propose', 10: 'hedge_count', 11: 'number_of_diff_dic_pos', 12: 'first_person_singular_count', 13: 'assertive_count', 14: 'liwc_informal', 15: 'pos_sentiment', 16: 'personal_concern', 17: 'third_person_singular', 18: 'politeness_please', 19: 'number_of_diff_dic_neg', 20: 'factive_count', 21: 'liwc

In [28]:
# Choose data based on certain characteristics
# Drop conversations with < 4 utterances
taken_data_all = {}
for dtype in full_data_all:
    print (dtype)
    taken_data = []
    for i in range(len(full_data_all[dtype])):
        if len(full_data_all[dtype][i]['utterance']) <= 4:
            continue
        taken_data.append(full_data_all[dtype][i])
    print ('Originally had : ', len(full_data_all[dtype]))
    print ('Taken          : ', len(taken_data))
    taken_data_all[dtype] = taken_data

train
Originally had :  5383
Taken          :  4828
test
Originally had :  656
Taken          :  567
dev
Originally had :  643
Taken          :  561


In [29]:
strategy_vector_data = {'train': taken_data_all['train'], 
                        'strategies2colid': strategies2colid, 
                        'dialacts2id': chosen_dial_act_mapping,
                        'test': taken_data_all['test'],
                        'valid' : taken_data_all['dev'],}

# Total conversations in data and zeros among them

In [32]:
total_conv = 0
zero_strategy_conv = 0
all_data_concatenated = full_data_all['train'] + full_data_all['test'] + full_data_all['dev'] 
for i in range(len(all_data_concatenated)):
    total_conv += all_data_concatenated[i]['strategies_vec'].shape[0]
    for j in range(all_data_concatenated[i]['strategies_vec'].shape[0]):
        if all(all_data_concatenated[i]['strategies_vec'][j] == 0):
            zero_strategy_conv += 1
print (f'Total conv are {total_conv}. Out of that {zero_strategy_conv} or {zero_strategy_conv / total_conv} are zero vectors.')

Total conv are 67891. Out of that 3060 or 0.04507224816249576 are zero vectors.


# Baseline Predict everyone as 0

In [33]:
for dtype in taken_data_all:
    print (dtype)
    total = 0
    ones = 0
    for i in range(len(taken_data_all[dtype])):
        temp = taken_data_all[dtype][i]['strategies_vec'].reshape(-1)
        total += len(temp)
        ones  += np.sum(temp)
    print (f'Total is {total}. Out of that {ones} are 1. Majority class would be : {(total-ones)/total}.')

train
Total is 1219138. Out of that 184512.0 are 1. Majority class would be : 0.8486537209077233.
test
Total is 140645. Out of that 16759.0 are 1. Majority class would be : 0.8808418358277934.
dev
Total is 142393. Out of that 21380.0 are 1. Majority class would be : 0.8498521696993532.


# Class weights (based on only training data)

In [34]:
# num_zeros / num_ones for each class
# sanity is that first class is 50/50
num_feats = taken_data_all['train'][0]['strategies_vec'].shape[1]
feat_freq = ddict(lambda : ddict(int))
for i in range(len(taken_data_all['train'])):
    vec = taken_data_all['train'][i]['strategies_vec']
    for j in range(num_feats):
        ones = np.sum(vec[:, j])
        zeros = vec.shape[0] - ones
        feat_freq[j][1] += ones
        feat_freq[j][0] += zeros
print (feat_freq)

defaultdict(<function <lambda> at 0x7f10e6be9dd0>, {0: defaultdict(<class 'int'>, {1: 23908.0, 0: 29098.0}), 1: defaultdict(<class 'int'>, {1: 197.0, 0: 52809.0}), 2: defaultdict(<class 'int'>, {1: 1691.0, 0: 51315.0}), 3: defaultdict(<class 'int'>, {1: 146.0, 0: 52860.0}), 4: defaultdict(<class 'int'>, {1: 877.0, 0: 52129.0}), 5: defaultdict(<class 'int'>, {1: 2842.0, 0: 50164.0}), 6: defaultdict(<class 'int'>, {1: 6036.0, 0: 46970.0}), 7: defaultdict(<class 'int'>, {1: 3618.0, 0: 49388.0}), 8: defaultdict(<class 'int'>, {1: 3155.0, 0: 49851.0}), 9: defaultdict(<class 'int'>, {1: 8333.0, 0: 44673.0}), 10: defaultdict(<class 'int'>, {1: 11995.0, 0: 41011.0}), 11: defaultdict(<class 'int'>, {1: 18178.0, 0: 34828.0}), 12: defaultdict(<class 'int'>, {1: 25496.0, 0: 27510.0}), 13: defaultdict(<class 'int'>, {1: 4384.0, 0: 48622.0}), 14: defaultdict(<class 'int'>, {1: 2374.0, 0: 50632.0}), 15: defaultdict(<class 'int'>, {1: 24464.0, 0: 28542.0}), 16: defaultdict(<class 'int'>, {1: 8910.0, 0

In [35]:
feature_weights = {}
for featid, freq in feat_freq.items():
    ones = freq[1]
    zeros = freq[0]
    weight = zeros / ones
    feature_weights[featid] = weight
print (feature_weights)

{0: 1.2170821482349004, 1: 268.06598984771574, 2: 30.34594914251922, 3: 362.05479452054794, 4: 59.440136830102624, 5: 17.65095003518649, 6: 7.781643472498343, 7: 13.650635710337204, 8: 15.800633914421553, 9: 5.360974438977559, 10: 3.4190079199666528, 11: 1.9159423478930575, 12: 1.0789927831816755, 13: 11.090784671532846, 14: 21.32771693344566, 15: 1.166693917593198, 16: 4.949046015712683, 17: 2.1785799952026865, 18: 147.47619047619048, 19: 4.201256010205083, 20: 14.728783382789317, 21: 20.321802091713597, 22: 9.97887323943662}


In [36]:
strategy_vector_data['feature_weights'] = feature_weights

In [37]:
# num_dial_acts = len(taken_data_all['dialacts2id'])
# da_freq = ddict(lambda: ddict(int)) # create again for chosen data
# for i in range(len(taken_data_all['train'])):
#     vec = taken_data_all['train'][i]['dial_acts_vec']

## NO NEED TO CREATE FEATURE WEIGHTS FOR DIAL ACTS. MY TRAINER WILL DO IT.

In [40]:
pickle.dump(strategy_vector_data, open('../../data/negotiation_data/strategy_vector_data.pkl', 'wb'))

# Dump WFST DATA (seq of bag)

In [42]:
for dtype in ['train', 'valid', 'test']:
    f = open(f'../wfst/finite_state_machine/seq_bag_strats_rj_{dtype}', 'w')
    for dat in strategy_vector_data[dtype]:
        for uttidx, svec in enumerate(dat['strategies_vec']):
            f.write('<'+''.join([str(int(ch)) for ch in svec[1:]])+'> ')
        f.write('\n')

# Dump WFST DATA (end form)

In [43]:
for dtype in ['train', 'valid', 'test']:
    f = open(f'../wfst/finite_state_machine/seq_end_strats_rj_{dtype}', 'w')
    for dat in strategy_vector_data[dtype]:
        for uttidx, svec in enumerate(dat['strategies_vec']):
            for colidx, ch in enumerate(svec[1:]):
                if ch == 0: continue
                strat = colid2strategies[colidx+1]
                f.write('<'+strat+'> ')
            f.write('<end> ')       # end of utt
        f.write('\n')

# Dump WFST DATA (Dialogue Acts)

In [44]:
for dtype in ['train', 'valid', 'test']:
    f = open(f'../wfst/finite_state_machine/seq_da_acts_rj_{dtype}', 'w')
    for dat in strategy_vector_data[dtype]:
        for uttidx, dial_act in enumerate(dat['dial_acts']):
            f.write('<'+dial_act+'> ')
        f.write('\n')

# Strategy Vector Data with WORD IDS

## Have to bert ids, word ids (based on some pretrained embedding - see yiheng's)

In [49]:
from transformers import BertTokenizer

In [50]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [51]:
dtypes = full_data_all.keys()
word2id =  {"[PAD]": 0, "<start>":1,"[CLS]":2,"[SEP]":3,"<offer>":4,"<reject>":5,"<accept>":6,"<quit>":7,"[UNK]":8}

In [52]:
# Set vocab using all training data
total = len(word2id) # curr count
for k, datapoint in enumerate(data_w_strategies['train']):
    print (f'{k}/{len(data_w_strategies["train"])}', end = '\r')
    scene = (datapoint["scenario"]["kbs"][1]["item"]["Category"] + " " + " ".join(datapoint["scenario"]["kbs"][1]["item"]["Description"])+ " " + datapoint["scenario"]["kbs"][1]["item"]["Title"])
    scene = utils.normalizeString(scene, {"price":None}, datapoint["scenario"], normalize_price=norm_price) 
    for word in scene.split():
        if word not in word2id:
            word2id[word] = total
            total += 1
            
    utterances = parse_example(Example.from_dict(datapoint, Scenario), price_tracker, templates)[2:]
    for i, uter in enumerate(datapoint['events']):
        if uter['action'] != 'message': continue
        tmp_dict = utterances[i]
        for word in utils.normalizeString(uter["data"], tmp_dict.lf.to_dict(), datapoint["scenario"], normalize_price=norm_price).split():
            if word not in word2id:
                word2id[word] = total
                total += 1
print("Vocab size : ", len(word2id))

Vocab size :  13119


In [53]:
strategy_vector_data['word2id'] = word2id

In [54]:
strategy_vector_data.keys()

dict_keys(['train', 'strategies2colid', 'dialacts2id', 'test', 'valid', 'feature_weights', 'word2id'])

In [55]:
cls_token_id = tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[CLS]'))
sep_token_id = tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))
for dtype in dtypes:
    print ()
    print (dtype)
    if dtype == 'dev': dtype = 'valid'
    for i, datapoint in enumerate(strategy_vector_data[dtype]):
        print (f'{i}/{len(strategy_vector_data[dtype])}', end = '\r')
        toks_space = [[word2id['[CLS]']] + [word2id.get(w, word2id['[UNK]']) for w in utt.split(' ')] + [word2id['[SEP]']] for utt in datapoint['utterance']]
        toks_bert  = [cls_token_id + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(utt)) + sep_token_id for utt in datapoint['utterance']]
        scene_space = [word2id.get(w, word2id['[UNK]']) for w in datapoint['scene'].split(' ')]
        scene_bert  = cls_token_id + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(datapoint['scene'])) + sep_token_id
        datapoint['toks_space'] = toks_space
        datapoint['toks_bert']  = toks_bert
        datapoint['scene_space'] = scene_space
        datapoint['scene_bert'] = scene_bert


train
4827/4828
test
566/567
dev


In [56]:
# Max scene seq
max_scene_seq = 0
for dtype in dtypes:
    if dtype == 'dev': dtype = 'valid'
    for datapoint in strategy_vector_data[dtype]:
        max_scene_seq = max(max_scene_seq, len(datapoint['scene_bert']))
print (max_scene_seq)

356


# add ratio bucket (obtained from only train data)

In [57]:
all_ratios = []
for datapoint in strategy_vector_data['train']:
    all_ratios.append(datapoint['ratio'])
print (len(all_ratios))

4828


In [58]:
# Split into 5
all_ratios = sorted(all_ratios)
bucket_size = int(len(all_ratios) * 0.2)
bucket1 = all_ratios[              : 1*bucket_size]
bucket2 = all_ratios[1*bucket_size : 2*bucket_size]
bucket3 = all_ratios[2*bucket_size : 3*bucket_size]
bucket4 = all_ratios[3*bucket_size : 4*bucket_size]
bucket5 = all_ratios[4*bucket_size :              ]
print (min(bucket1), max(bucket1))
print (min(bucket2), max(bucket2))
print (min(bucket3), max(bucket3))
print (min(bucket4), max(bucket4))
print (min(bucket5), max(bucket5))

-99999 -0.2
-0.2 0.16666666666666666
0.16666666666666666 0.4
0.4 0.6052631578947368
0.6052631578947368 254.8139534883721


In [59]:
for dtype in dtypes:
    print (dtype)
    if dtype == 'dev': dtype = 'valid'
    for datapoint in strategy_vector_data[dtype]:
        ratio = datapoint['ratio']
        if   ratio <= max(bucket1): bucket = 0
        elif ratio <= max(bucket2): bucket = 1
        elif ratio <= max(bucket3): bucket = 2
        elif ratio <= max(bucket4): bucket = 3
        else:                       bucket = 4
        datapoint['ratio_bucket'] = bucket
print (strategy_vector_data['train'][2])

train
test
dev
{'agent_list': [-1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0], 'utterance': ['<start>', 'i would love to buy', 'sure ! what s your price ?', 'im on a budget so i could do <price>_-0.1', 'how about <price>_-0.0 and i ll wave the deposit .', 'i will take it', 'great !', '<offer>', '<offer>', '<accept>', '<accept>'], 'strategies': [['<start>'], ['buyer_pos_sentiment', 'first_person_singular_count_buyer', 'hedge_count_buyer'], ['seller_pos_sentiment', 'number_of_diff_dic_neg', 'assertive_count_seller', 'personal_concern_seller'], ['first_person_singular_count_buyer', 'buyer_propose', 'hedge_count_buyer'], ['first_person_singular_count_seller', 'number_of_diff_dic_neg', 'hedge_count_seller', 'personal_concern_seller'], ['first_person_singular_count_buyer', 'third_person_singular_buyer'], ['seller_pos_sentiment'], ['seller_pos_sentiment'], ['seller_pos_sentiment'], ['seller_pos_sentiment'], ['seller_pos_sentiment']], 'ratio': 0.037037037037037035, 'dial_acts': ['<start>', 'intro', 'inquir

In [60]:
#pickle.dump(strategy_vector_data, open('../../../data/negotiation_data/data/strategy_vector_data_FULL.pkl', 'wb'))
pickle.dump(strategy_vector_data, open('../../data/negotiation_data/strategy_vector_data_FULL_Yiheng.pkl', 'wb'))

In [62]:
for w in strategy_vector_data['word2id']:
    if '<price' in w:
        print(w)

<price>_-0.2
<price>_0.0
<price>_0.4
<price>_0.3
<price>_-0.1
<price>_-0.0
<price>_-0.3
<price>_0.6
<price>_0.5
<price>_-1.5
<price>_0.7
<price>_-0.6
<price>_-2.0
<price>_-1.2
<price>_0.8
<price>_-0.7
<price>_2.0
<price>_0.2
<price>_-1.9
<price>_-0.4
<price>_1.1
<price>_0.9
<price>_1.0
<price>_-0.5
<price>_0.1
<price>_1.8
<price>_1.2
<price>_-1.0
<price>_1.4
<price>_-0.8
<price>_-0.9
<price>_-1.1
<price>_-1.3
<price>_1.5
<price>_-1.4
<price>_1.6
<price>_1.7
<price>_1.3
<price>_-1.7
<price>_1.9
<price>_-1.6
<price>_-1.8
